# Chatbot for E-commerce

## 1. Data Analysis

In [98]:
import json
import nltk

words=[]
classes = []
documents = []

ignore_words = ['?', '!', ".", ",", "'s"]

data_file = open('intents.json').read()
intents = json.loads(data_file)

In [99]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Nice to meet you.',
    'Hello, Good to see you.',
    'Top of the morning.'],
   'context': ['']},
  {'tag': 'introduce',
   'patterns': ['What are you?',
    'Who are you?',
    'Can you introduce yourself?',
    "What's your name?"],
   'responses': ['My name is ECOM. I am your Online Shopping Assistant.'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye', 'Till next time'],
   'responses': ['See you! Thanks for shopping with us.',
    'Have a nice day',
    'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'buybooks',
   'patterns': ['I 

In [100]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [101]:
print("Classes: ", classes, "\n")
print("Words: ", words, "\n")
print("Documents: ", documents, "\n")

Classes:  ['greeting', 'introduce', 'goodbye', 'thanks', 'buybooks', 'payment', 'noanswer'] 

Words:  ['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'What', 'are', 'you', '?', 'Who', 'are', 'you', '?', 'Can', 'you', 'introduce', 'yourself', '?', 'What', "'s", 'your', 'name', '?', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Till', 'next', 'time', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Awesome', ',', 'thanks', 'Thanks', 'for', 'helping', 'me', 'I', 'want', 'to', 'buy', 'a', 'romantic', 'comedy', 'genre', 'book', 'I', 'want', 'to', 'buy', 'the', 'newest', 'novel', 'you', 'have', 'What', 'books', 'do', 'you', 'have', '?', 'Do', 'you', 'sell', 'books', '?', 'Can', 'you', 'recommend', 'me', 'some', 'books', '?', 'How', 'can', 'i', 'pay', '?', 'I', 'want', 'to', 'pay', 'for', 'this', 'I', 'want', 'to', 'order', 'How', 'do', 'i', 'order', '?', 'How', 'much', 'money', 'does', 'this', 'cost', '?', 'Check', 'out', 'please'] 

D

## 2. Preprocess data


In [102]:
import nltk
from nltk.stem import WordNetLemmatizer
import random
import numpy as np

lemmatizer = WordNetLemmatizer()

In [103]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

print(words)

['a', 'anyone', 'are', 'awesome', 'book', 'buy', 'bye', 'can', 'check', 'comedy', 'cost', 'day', 'do', 'doe', 'for', 'genre', 'good', 'goodbye', 'have', 'hello', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'introduce', 'is', 'later', 'me', 'money', 'much', 'name', 'newest', 'next', 'novel', 'order', 'out', 'pay', 'please', 'recommend', 'romantic', 'see', 'sell', 'some', 'thank', 'thanks', 'that', 'the', 'there', 'this', 'till', 'time', 'to', 'want', 'what', 'who', 'you', 'your', 'yourself']


In [104]:
# sort classes
classes = sorted(list(set(classes)))
classes

['buybooks',
 'goodbye',
 'greeting',
 'introduce',
 'noanswer',
 'payment',
 'thanks']

In [105]:
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


61 unique lemmatized words ['a', 'anyone', 'are', 'awesome', 'book', 'buy', 'bye', 'can', 'check', 'comedy', 'cost', 'day', 'do', 'doe', 'for', 'genre', 'good', 'goodbye', 'have', 'hello', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'introduce', 'is', 'later', 'me', 'money', 'much', 'name', 'newest', 'next', 'novel', 'order', 'out', 'pay', 'please', 'recommend', 'romantic', 'see', 'sell', 'some', 'thank', 'thanks', 'that', 'the', 'there', 'this', 'till', 'time', 'to', 'want', 'what', 'who', 'you', 'your', 'yourself']


In [106]:
with open("words_data.json", "w") as words_data:
    json.dump(words, words_data)

In [107]:
with open("classes_data.json", "w") as classes_data:
    json.dump(classes, classes_data)

## 3. Create training and testing data

In [108]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

In [109]:
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [110]:
for t in training:
    print(t)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0]]
[[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 0, 0]]
[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [111]:
output_empty

[0, 0, 0, 0, 0, 0, 0]

In [112]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype=object)
# create train and test lists. X - patterns, Y - intents
X_train = list(training[:,0])
y_train = list(training[:,1])
print("Training data created")

Training data created


In [113]:
y_train

[[1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1],
 [0, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1],
 [0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1]]

In [114]:
for i in range(len(X_train)):
    print(X_train[i], y_train[i])

[1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0] [1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0] [1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0] [0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1

## 4 Build the Sequential Model

In [115]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras.utils import plot_model

In [116]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))

In [117]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [118]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 128)               7936      
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 7)                 455       
Total params: 16,647
Trainable params: 16,647
Non-trainable params: 0
_________________________________________________________________


In [119]:
#fitting and saving the model 
hist = model.fit(np.array(X_train), np.array(y_train), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)
print("model created")

Epoch 1/200
7/7 [==============================] - 0s 5ms/step - loss: 1.9115 - accuracy: 0.2500
Epoch 2/200
7/7 [==============================] - 0s 1ms/step - loss: 1.9254 - accuracy: 0.1250
Epoch 3/200
7/7 [==============================] - 0s 743us/step - loss: 1.8104 - accuracy: 0.2812
Epoch 4/200
7/7 [==============================] - 0s 997us/step - loss: 1.7383 - accuracy: 0.3125
Epoch 5/200
7/7 [==============================] - 0s 1ms/step - loss: 1.6459 - accuracy: 0.3750
Epoch 6/200
7/7 [==============================] - 0s 997us/step - loss: 1.5176 - accuracy: 0.4062
Epoch 7/200
7/7 [==============================] - 0s 1ms/step - loss: 1.6046 - accuracy: 0.3438
Epoch 8/200
7/7 [==============================] - 0s 997us/step - loss: 1.4696 - accuracy: 0.5625
Epoch 9/200
7/7 [==============================] - 0s 998us/step - loss: 1.4420 - accuracy: 0.5000
Epoch 10/200
7/7 [==============================] - 0s 1ms/step - loss: 1.4088 - accuracy: 0.5625
Epoch 11/200
7/7 [=

7/7 [==============================] - 0s 777us/step - loss: 0.0597 - accuracy: 1.0000
Epoch 84/200
7/7 [==============================] - 0s 855us/step - loss: 0.0495 - accuracy: 1.0000
Epoch 85/200
7/7 [==============================] - 0s 856us/step - loss: 0.0694 - accuracy: 1.0000
Epoch 86/200
7/7 [==============================] - 0s 751us/step - loss: 0.0814 - accuracy: 1.0000
Epoch 87/200
7/7 [==============================] - 0s 1ms/step - loss: 0.0792 - accuracy: 0.9688
Epoch 88/200
7/7 [==============================] - 0s 855us/step - loss: 0.0495 - accuracy: 1.0000
Epoch 89/200
7/7 [==============================] - 0s 1ms/step - loss: 0.1364 - accuracy: 0.9688
Epoch 90/200
7/7 [==============================] - 0s 855us/step - loss: 0.1074 - accuracy: 0.9688
Epoch 91/200
7/7 [==============================] - 0s 854us/step - loss: 0.2105 - accuracy: 0.9688
Epoch 92/200
7/7 [==============================] - 0s 570us/step - loss: 0.1268 - accuracy: 0.9375
Epoch 93/200
7/7 

7/7 [==============================] - 0s 709us/step - loss: 0.0293 - accuracy: 1.0000
Epoch 165/200
7/7 [==============================] - 0s 855us/step - loss: 0.0745 - accuracy: 0.9688
Epoch 166/200
7/7 [==============================] - 0s 855us/step - loss: 0.0290 - accuracy: 1.0000
Epoch 167/200
7/7 [==============================] - 0s 715us/step - loss: 0.0433 - accuracy: 1.0000
Epoch 168/200
7/7 [==============================] - 0s 966us/step - loss: 0.0369 - accuracy: 1.0000
Epoch 169/200
7/7 [==============================] - 0s 570us/step - loss: 0.0089 - accuracy: 1.0000
Epoch 170/200
7/7 [==============================] - 0s 570us/step - loss: 0.0290 - accuracy: 1.0000
Epoch 171/200
7/7 [==============================] - 0s 857us/step - loss: 0.0479 - accuracy: 1.0000
Epoch 172/200
7/7 [==============================] - 0s 715us/step - loss: 0.0274 - accuracy: 1.0000
Epoch 173/200
7/7 [==============================] - 0s 950us/step - loss: 0.0821 - accuracy: 0.9688
Epoc

In [120]:
# plot_model(model, to_file = "model.png", show_shapes=True)

## Testing

In [95]:
def clean(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words):
    # tokenize the pattern
    sentence_words = clean(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.5
#     print(res)

    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [96]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = predict_class(text, model)
    predict_intent = ints[0]['intent']
    # print(ints)
    res = getResponse(ints, intents)
    
    return res, ints

In [97]:
while(True):
    w = str(input())
    print("You: ", w)
    res = chatbot_response(w)
    print("Bot: ",res)
    

Hello
You:  Hello
Bot:  ('Top of the morning.', [{'intent': 'greeting', 'probability': '0.9997243'}])
Who are you?
You:  Who are you?
Bot:  ('My name is ECOM. I am your Online Shopping Assistant.', [{'intent': 'introduce', 'probability': '0.9999962'}])
I want to buy a new books
You:  I want to buy a new books
Bot:  (' Here are our list of products.', [{'intent': 'buybooks', 'probability': '0.99999976'}])
What novels do you have?
You:  What novels do you have?
Bot:  (' Here are our list of products.', [{'intent': 'buybooks', 'probability': '0.99983346'}])
Thanks
You:  Thanks
Bot:  ('Any time!', [{'intent': 'thanks', 'probability': '0.99986804'}])
I want to pay for this book
You:  I want to pay for this book
Bot:  (' Please fill in your payment form.', [{'intent': 'payment', 'probability': '0.9996896'}])
can i pay?
You:  can i pay?
Bot:  (' Please fill in your payment form.', [{'intent': 'payment', 'probability': '0.99993527'}])

You:  
Bot:  ("Sorry, can't understand you", [{'intent': '

KeyboardInterrupt: Interrupted by user